In [ ]:
#import necessary modules
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal, gdalconst
from rasterio.plot import show
import earthpy.spatial as es
import earthpy.plot as ep
from numpy import savetxt
from numpy import loadtxt

In [ ]:
#Set imagery as variables
asi = #'Path_to_SingleDateImage_Amsterdam.tif'
ami = #'Path_to_MedianImage_Amsterdam.tif'
msi = #'Path_to_SingleDateImage_Milano.tif'
mmi = #'Path_to_MedianImage_Milano.tif'
bsi = #'Path_to_SingleDateImage_Budapest.tif'
bmi = #'Path_to_MedianImage_Budapest.tif'
filist = [asi, ami, msi, mmi, bsi, bmi]

In [ ]:
#Create function that reads imagery variable and seperates bands
def read_data(inraster):
    def convert(img_g, target_type_min, target_type_max, target_type):
        imin = np.amin(img_g)
        imax = np.amax(img_g)
        a = (target_type_max - target_type_min) / (imax - imin)
        b = target_type_max - a * imax
        new_img = (a * img_g + b).astype(target_type)
        return new_img
    
    img = gdal.Open(inraster, gdal.GA_ReadOnly) 
    bands = [img.GetRasterBand(i).ReadAsArray() for i in range(1, img.RasterCount + 1)]
    img = np.array(bands)
    img = img[0:12,:,:]
    img = np.transpose(img, [1, 2, 0])
    imgblue = img[:,:,[1]]
    imgblue = imgblue[:, :, 0]
    imggreen = img[:,:,[2]]
    imggreen = imggreen[:, :, 0]
    imgred = img[:,:,[3]]
    imgred = imgred[:, :, 0]
    imgnir = img[:,:,[7]]
    imgnir = imgnir[:, :, 0]
    img_rgb = img[:,:,[3,2,1]]
    blueband = convert(imgblue, 0, 256, np.uint8)
    greenband = convert(imggreen, 0, 256, np.uint8)
    redband = convert(imgred, 0, 256, np.uint8)
    nirband = convert(imgnir, 0, 256, np.uint8)
    return imgblue, imggreen, imgred, imgnir

In [ ]:
%%time
#https://github.com/berkesenturk/Remote-Sensing-Sentinel-2-EO-Data-Masking-Algorithm/blob/main/utils.py
NDWIlist = {}
x = 0
for img in sublist:
    x += 1
    blueband, green_band, redband, nir_band = read_data(img)
    np.seterr(divide='ignore', invalid='ignore')
    ndwi = (nir_band - green_band) / (nir_band + green_band)
    ndwi= np.nan_to_num(ndwi)
    NDWIlist[x] = ndwi

In [ ]:
#save NDVI as CSV files
ndwiasi = np.savetxt('ndwiasi.csv', NDWIlist[1], delimiter=',')
ndwiami = np.savetxt('ndwiami.csv', NDWIlist[2], delimiter=',')
ndwimsi = np.savetxt('ndwimsi.csv', NDWIlist[3], delimiter=',')
ndwimmi = np.savetxt('ndwimmi.csv', NDWIlist[4], delimiter=',')
ndwibsi = np.savetxt('ndwibsi.csv', NDWIlist[5], delimiter=',')
ndwibmi = np.savetxt('ndwibmi.csv', NDWIlist[6], delimiter=',')

#load file example
#load NDWI
ndwiasi = loadtxt('ndwiasi.csv', delimiter=',')

In [ ]:
#Plot NDWI results
ndwi = ndwiasi
titles = #'Insert Title'
ep.plot_bands(ndwi, cmap="RdYlGn", cols=1, title=titles, vmin=0, vmax=1)